In [1]:
from sagemaker import get_execution_role
from sagemaker.session import Session

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = Session().default_bucket()

# Location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://{}/machinelearning-rd/customcode/tensorflow_iris'.format(bucket)

# Location where results of model training are saved.
model_artifacts_location = 's3://{}/machinelearning-rd/artifacts'.format(bucket)

#IAM execution role that gives SageMaker access to resources in your AWS account.
role = get_execution_role()

In [7]:
from sagemaker.tensorflow import TensorFlow

iris_estimator = TensorFlow(entry_point='iris_dnn_classifier.py',
                            role=role,
                            framework_version='1.12.0',
                            output_path=model_artifacts_location,
                            code_location=custom_code_upload_location,
                            train_instance_count=1,
                            train_instance_type='ml.c4.xlarge',
                            training_steps=1000,
                            evaluation_steps=100)

W0809 10:16:05.410323 140176566253376 estimator.py:286] tensorflow py2 container will be deprecated soon.


In [9]:
%%time
import boto3

# use the region-specific sample data bucket
region = boto3.Session().region_name
train_data_location = 's3://sagemaker-sample-data-{}/tensorflow/iris'.format(region)

iris_estimator.fit(train_data_location)

2019-08-09 10:21:15 Starting - Starting the training job...
2019-08-09 10:21:16 Starting - Launching requested ML instances......
2019-08-09 10:22:19 Starting - Preparing the instances for training...
2019-08-09 10:23:11 Downloading - Downloading input data...
2019-08-09 10:23:34 Training - Training image download completed. Training in progress.
2019-08-09 10:23:34,286 INFO - root - running container entrypoint
2019-08-09 10:23:34,286 INFO - root - starting train task
2019-08-09 10:23:34,299 INFO - container_support.training - Training starting
2019-08-09 10:23:37,244 INFO - tf_container - ----------------------TF_CONFIG--------------------------
2019-08-09 10:23:37,244 INFO - tf_container - {"environment": "cloud", "cluster": {"master": ["algo-1:2222"]}, "task": {"index": 0, "type": "master"}}
2019-08-09 10:23:37,244 INFO - tf_container - ---------------------------------------------------------
2019-08-09 10:23:37,244 INFO - tf_container - creating RunConfig:
2019-08-09 10:23:37,244

In [10]:
%%time
iris_predictor = iris_estimator.deploy(initial_instance_count=1,
                                       instance_type='ml.m4.xlarge')

W0809 10:28:47.844547 140176566253376 model.py:103] The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


--------------------------------------------------------------------------------------------------!CPU times: user 509 ms, sys: 18.1 ms, total: 527 ms
Wall time: 8min 16s


In [11]:
iris_predictor.predict([6.4, 3.2, 4.5, 1.5])

{'result': {'classifications': [{'classes': [{'label': '0',
      'score': 3.6902718420606107e-05},
     {'label': '1', 'score': 0.9997839331626892},
     {'label': '2', 'score': 0.00017915625358000398}]}]},
 'model_spec': {'name': 'generic_model',
  'version': {'value': 1565346226},
  'signature_name': 'serving_default'}}

In [12]:
import sagemaker

sagemaker.Session().delete_endpoint(iris_predictor.endpoint)

In [13]:
print(iris_predictor.endpoint)

sagemaker-tensorflow-2019-08-09-10-21-15-265
